# 2D Example of a electrode facing a charged rod

# Geometry

We will define a 2D region where the potential is fixed (rect_pot), one where the charge is fixed to zero (rect_out) 
and a third where there is a point charge (rect_int). 

In [ ]:
import poisson

l = 2
L = 2
W = 40

rect_int = poisson.continuous.shapes.Rectangle(length=(l, l), corner=(-l/2, -l/2))
rect_out = poisson.continuous.shapes.Rectangle(length=(W, L + W/2), corner=(-W/2, -L))
rect_pot = poisson.continuous.shapes.Rectangle(length=(W, 1.01), corner=(-W/2, -L - 1))

con_geo = poisson.ContinuousGeometry(voltage = {'electrode':rect_pot}, 
                                     charge = {'insulator': rect_out - rect_int - rect_pot, 'point charge': rect_int})

We will define a mesh that is very fine around the charge and coarser elsewhere

In [ ]:
bbox =  [-W/2, W/2, -L - 5, W/2]
grid = poisson.GridBuilder(meshs=[(bbox, 0.1, rect_out, 0),
                                  (bbox, 0.01, rect_int, 1),
                                  (bbox, 0.05, rect_pot, 2)])

In [ ]:
# Plot them
con_geo_plot = con_geo.plot(plot_type='2D', bbox=bbox) # This returns a function that when called creates a plot

con_geo_plot(variable=0) # variable must be 0

# Discrete Poisson 

In [ ]:
dis_poisson = poisson.DiscretePoisson(
    con_geo, grid=grid, selection={'Voronoi':[['voltage', '*']]})


# Linear Problem

In this problem one must define a charge at point_charge. 

This can be done in two ways: 

- Impose the total number of charge carriers at the volume element: e.g. 2  (which corresponds to 2 electrons, i.e. a charge of 2 * 1.69e-19 C)
- Impose a density. This part is trickier, as if the density is defined in 3D it must be transformed to a 2D density where the density along the dimension that was lost is considered invariant. In order to do so one must define the 3D density in $\text{nm}^{-3}$ and then divide the density by the length of the system along the invariant dimension. FOr a more comprehensive explanation see the Report. 

In order to associate a voltage or charge value to a region one can do that in multiple ways: 

- Define a tuple containing :
  - A Shape or function as first element and a constant value as second                                                             
  - Shape or function in first element and a function as second element describing the variation of the value as a function of the coordinates. Thus the function must take as input a set of coordinates and return a float. 
  - A numpy array containing the index of the points where the voltage or charge is fixed as a the first element and a numpy array with their respective values as a second element. 
        
Similarly to what was done earlier, a list of tuples must be given as voltage or charge. 

In the follwing example the voltage will be defined with (shape, float) and charge with (np.array, np.array). The charge will be given in numbers of charge carriers per volume element. An example where the density is given is example_dopant. 

In [ ]:
from scipy.spatial import cKDTree
import numpy as np
# Select the index of the coordianate we are looking for
tree =  cKDTree(dis_poisson.mesh.points)
part_pos_y = 0.0
part_pos_x = 0.0
t, index = tree.query([part_pos_x,  part_pos_y])

# Create and solve the linear problem

linear_problem = poisson.LinearProblem( 
            dis_poisson, is_charge_density=False,
            voltage_val = [(rect_pot, 0.0)],
            charge_val=[(np.array([index]), np.array([1]))]) # we put one electron per volume element


#  Visualization

Before plotting the results one can visualize the charge density in every subregion, 
this can be done by using poisson.tools.regions_summary_charge 

In [ ]:
hyper_volume, charges, charge_tot = poisson.tools.regions_summary_charge(lin_prob_inst=linear_problem, verbose=True)

In [ ]:
from scipy import constants

# Analytical calculation :
A = -constants.elementary_charge / (2 * np.pi * constants.epsilon_0 * 1e-9)

y = np.linspace(-W, W, 10000)
r_0 = -2*L
V = -A * (np.log(np.abs(y)) - np.log((np.abs(y - r_0))))

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plot_voltage_2d, plot_charge_2d = linear_problem.plot_cut_2d(direction=2)
plot_voltage_2d(0, colorbar_label=r'Voltage (V)')
plot_charge_2d(0, colorbar_label=r'Charge density($\#.nm^{{{-2}}}$)')
plt.show()

# bbox=[(-W/2, W/2, 3000)]
plot_voltage, plot_charge = linear_problem.plot_cut_1d(
        directions=(0,), bbox='default', npoints=2000)

fig2 = plt.figure()
ax_voltage = fig2.add_subplot(111)

t, data_volt = plot_voltage(
        (0, ), ax = ax_voltage, marker='.', color='k',
        label='Voltage simulation', linestyle='None')

ax_voltage.set_xlabel('{0}(nm)'.format('y'))
ax_voltage.set_ylabel('Voltage(V)')
ax_voltage.plot(y, V, 'g-', label='Voltage analytical')
ax_voltage.legend(loc='upper center')

# ax_charge = ax_voltage.twinx()
fig2 = plt.figure()
ax_charge = fig2.add_subplot(111)

tt, data_charge = plot_charge(
        (0, ), ax = ax_charge, marker='.', color='b',
        label='Charge simulation', linestyle='None')

ax_charge.set_xlabel('{0}(nm)'.format('y'))
ax_charge.set_ylabel(r'Charge density $(\#.nm^{{{-2}}})$')
ax_charge.legend(loc='lower center')

plt.show()